# Задача 1

Для начала, чтобы искать какие-то вероятности, нужно предложить модель данных. Так как известно всего 5 измерений, какие-то тесты на нормальность проводить бесполезно. Но для погрешности приборов предполагать нормальное распределение с неизвестной дисперсией - разумно. Так и поступим.

Есть два известных подхода в статистике - frequentist(частотный) и Байесовский. Частотный обычно преподается в университетах, именно по нему читаются лекции, проводятся курсы и т.д, поэтому есть подозрение, что вопрос должен был быть именно про него, хотя в этой формулировке он не имеет большого смысла. В любом случае, разберем сначала этот подход, потом опишу Байесовский.

## Frequentist

В частотном подходе принято считать, что параметры модели - не случайные величины, а просто неизвестные числа. Поэтому говорить о вероятности события $\mu < 5$ нельзя. Если true $\mu$ меньше 5, то вероятность 1, иначе - 0. Однако мы можем строить гипотезы, проверять их.
Допустим, у нашего прибора нормальное распределение.
$$X_1, \ldots, X_5 \overset{i.i.d}{\sim} \mathcal{N}(\mu, \sigma^2)$$
$$\overline{X} = \frac{1}{5}\sum\limits_{i=1}^{5}{X_i} = 4.95$$
$$S_n = \frac{1}{4}\sum\limits_{i=1}^{5}{(X_i - \overline{X})^2} = 0.212$$
Тогда
$$ \sqrt{5} \frac{\overline{X} - \mu}{\sqrt{S_n}} \sim t_{4}$$
Это т-распределение со степенью свободы $(n-1)$. Можно этим фактом пользоваться следующим образом - мы можем отвергнуть какое-то $\mu$, если получившееся значение будет "не характерным" для $t_{n-1}$ распределения. Это t-test. Рассмотрим значения тестовой статистики для $\mu < 4$, $\mu < 5$.

In [6]:
from scipy.stats import ttest_1samp
X = [5.2, 4.3, 4.7, 4.9, 5.5]
print(ttest_1samp(X, 4))
print(ttest_1samp(X, 5))

Ttest_1sampResult(statistic=4.467914966843415, pvalue=0.01109246432618955)
Ttest_1sampResult(statistic=-0.388514344942906, pvalue=0.7174283599144878)


Получается, что для $\mu < 4$ значение тестовой статистики будет принимать значения $> 4.467$ - это маловероятные значения. В этой реализации т-тест двухсторонний, а нам нужен односторонний, поэтому для $\mu < 4$ p-value должно быть еще в два раза меньше, то есть $\approx 0.005$ - эту гипотезу разумно отвергнуть. 

Для $\mu < 5$ значение статистики будут больше $-0.38$, а p-value тогда получается $1 - 0.7174 / 2 = 0.64$ - разумеется мы не можем отвергнуть эту гипотезу.

## Байесовский подход

В Байесовской статистике как раз рассматривается распределение параметра модели при условии полученных данных. То есть, мы хотим найти

$$P(\mu ~|~ X_1, \ldots, X_5, \sigma^2) \propto P(X_1, \ldots, X_5 ~|~ \mu, \sigma^2) P(\mu ~|~ \sigma^2)$$

Далее нужно выбрать какое-то prior distribution (то есть $P(\mu ~|~ \sigma^2)$, так как $\sigma^2$ нас не интересует, и тогда $P(\mu ~|~ X_1 \ldots, X_n)$ мы найдем как $\int{P(\mu, \sigma^2 ~|~ X_1, \ldots) \mathrm{d}\sigma^2}$). Не желательно отдавать предпочтений каким-то $\mu$, поэтому prior у нас константный - это improper prior, так как интеграл $\int_{-\infty}^{+\infty}{const} = \infty$.

Теперь мы свели задачу к нахождению posterior distribution при нормальных данных и константном prior. Эта задача сложная, но результат у нее интересный. В [данной статье](http://www.stat.columbia.edu/~fwood/Teaching/w4315/Fall2009/nuisance_parameters.pdf) эта задача решается, хотя полного вывода в ней нет - написано, что алгебраические выкладки сложные и сделана ссылка на другую статью.
Итог следующий - если выбрать prior $P(\mu ~|~ \sigma^2) = (\sigma^2)^{-1}$, то posterior distribution получается из следующего доказанного факта:

$$\sqrt{n} \frac{\overline{X} - \mu}{\sqrt{S_n}} ~|~ X_1, \ldots, X_n \sim t_{n-1}$$

Таким образом мы опять пришли к т-распределению, но уже через Байесовскую статистику, что очень круто. Ответ можно получить из полученных ранее p-values:

$$P(\mu < 4 ~|~ X_1, \ldots, X_5) \approx 0.005 , \quad P(\mu < 5 ~|~ X_1, \ldots, X_5) \approx 0.64$$


# Задача 2

Тут можно применить формулу Байеса. Пусть $A_1$ - событие, что первый лучник попал, $A_2$ - что второй лучник попал, $B$ - что попал только один из лучников.

$$P(A_1) = 0.7, \quad P(A_2) = 0.55, \quad P(A_1 ~|~ B) ~=~ ?, \quad P(A_2 ~|~ B) ~=~ ?$$

Тогда

$$P(A_1 ~|~ B) = \frac{P(B ~|~ A_1) P(A_1)}{P(B)}, \quad P(A_2 ~|~ B) = \frac{P(B ~|~ A_2) P(A_2)}{P(B)}$$

$$P(B ~|~ A_1) = P(\neg A_2) = 0.45, \quad P(B ~|~ A_2) = P(\neg A_1) = 0.3, \quad P(B) = P(A_1)P(\neg A_2) + P(\neg A_1)P(A_2) = 0.7 \cdot 0.45 + 0.3 \cdot 0.55 = 0.55$$

Отсюда получаем ответ:

$$P(A_1 ~|~ B) = \frac{0.45 \cdot 0.7}{0.55} = 0.65625, \quad P(A_2 ~|~ B) = \frac{0.3 \cdot 0.55}{0.55} = 0.34375$$


# Задача 4

Задача очень абстрактная, поэтому существует много причин, почему получается такая неувязка. Это может быть ошибка в измерениях, может быть погрешность, могут какие-то внешние факторы. Ниже я распишу возможные варианты.

1) Проблемы с замером. Если это тахометрический счетчик воды, может быть, что-то мешает вращению - следовательно, показатели этого счетчика занижаются. В таком случае, на вход на самом деле идет 95, на выходе все меряется верно - 35 и 60.

2) Есть задержка между замером показаний на входе и выходе. К примеру, если воду на входе начали выключать, то счетчик на входе мог зафиксировать это раньше, чем счетчики на выходе. В таком случае, все показатели верные, и спустя какое-то время сумма входных и выходных значений снова сравняется. Тогда реальные значения и есть 90, 35, 60, но в ближайшее время они будут меняться.

3) Может быть, в узле как-то может накапливаться вода, тогда когда ее "выпускают", на выходе скорость и расход будет больше. В таком случае опять-таки в узлах правильные значения - 90, 35, 60, но через какое-то время на выходе значения уменьшатся.

4) Возможно, у измерений есть огромная погрешность. Может, прибор настроен на замер значений огромного порядка и поэтому погрешность в 5 литров - не очень большая.